In [1]:
import sys
sys.path.insert(0, '../..')
import hv_cancer_modules as hvc

ModuleNotFoundError: No module named 'hv_cancer_modules'

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pooch
import scanpy as sc
import anndata as ad

import pandas as pd
import hvplot.pandas #noqa
import datashader as ds
import colorcet as cc

import holoviews as hv
from holoviews.operation import datashader as hd

hv.extension('bokeh')

In [ ]:
sc.settings.set_figure_params(dpi=50, facecolor="white")

The data used in this basic preprocessing and clustering tutorial was collected from bone marrow mononuclear cells of healthy human donors and was part of [openproblem's NeurIPS 2021 benchmarking dataset](https://openproblems.bio/competitions/neurips_2021/) {cite}`Luecken2021`. The samples used in this tutorial were measured using the 10X Multiome Gene Expression and Chromatin Accessability kit. 


We are reading in the count matrix into an [AnnData](https://anndata.readthedocs.io/en/latest/tutorials/notebooks/getting-started.html) object, which holds many slots for annotations and different representations of the data.

In [ ]:
EXAMPLE_DATA = pooch.create(
    path=pooch.os_cache("scverse_tutorials"),
    base_url="doi:10.6084/m9.figshare.22716739.v1/",
)
EXAMPLE_DATA.load_registry_from_doi()

In [ ]:
%%time

samples = {
    "s1d1": "s1d1_filtered_feature_bc_matrix.h5",
    "s1d3": "s1d3_filtered_feature_bc_matrix.h5",
}
adatas = {}

for sample_id, filename in samples.items():
    path = EXAMPLE_DATA.fetch(filename)
    sample_adata = sc.read_10x_h5(path)
    sample_adata.var_names_make_unique()
    adatas[sample_id] = sample_adata

adata = ad.concat(adatas, label="sample")
adata.obs_names_make_unique()
print(adata.obs["sample"].value_counts())
adata

The data contains ~8,000 cells per sample and 36,601 measured genes. We'll now investigate these with a basic preprocessing and clustering workflow.

## Quality Control

The scanpy function {func}`~scanpy.pp.calculate_qc_metrics` calculates common quality control (QC) metrics, which are largely based on `calculateQCMetrics` from scater {cite}`McCarthy2017`. One can pass specific gene population to {func}`~scanpy.pp.calculate_qc_metrics` in order to calculate proportions of counts for these populations. Mitochondrial, ribosomal and hemoglobin genes are defined by distinct prefixes as listed below. 

In [ ]:
# mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes
adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True
)

One can now inspect violin plots of some of the computed QC metrics:

* the number of genes expressed in the count matrix
* the total counts per cell
* the percentage of counts in mitochondrial genes

In [ ]:
adata

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True,
)

With HoloViews, using the holoviews-anndata data interface:

In [ ]:
violins = []
for i in ["obs.n_genes_by_counts", "obs.total_counts", "obs.pct_counts_mt"]:
    violins.append(
        hv.Violin(adata, vdims=i).opts(
            ylabel='Value',
            title=i.split('.')[-1], # drop the 'obs.'
            show_grid=True,
            ylim=(0,None),
        )
    )
hv.Layout(violins).opts(axiswise=True)

Additionally, it is useful to consider QC metrics jointly by inspecting a scatter plot colored by `pct_counts_mt`. 

In [ ]:
sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

With HoloViews:

In [ ]:
scatter = (hv.Scatter(adata, "obs.total_counts", ["obs.n_genes_by_counts", "obs.pct_counts_mt"])
    .opts(cmap="Viridis",
          color="obs.pct_counts_mt",
          colorbar=True,
          width=400,
          tools=['hover'],
          show_grid=True,
          title='pct_counts_mt',
))

# Optionally, apply rasterization/datashading
rast_scatter = hd.rasterize(scatter)
rast_scatter.opts(
    cmap='Viridis',
    xticks=[0, 50000, 100000, 150000],
    padding=.05,
    show_grid=True,
    title='pct_counts_mt',
    colorbar=True,
    aspect=1,
    frame_width=300,
    tools=['hover'],
    cnorm='eq_hist', # eq_hist normalizes the color range
)

Based on the QC metric plots, one could now remove cells that have too many mitochondrial genes expressed or too many total counts by setting manual or automatic thresholds. However, sometimes what appears to be poor QC metrics can be driven by real biology so we suggest starting with a very permissive filtering strategy and revisiting it at a later point. We therefore now only filter cells with less than 100 genes expressed and genes that are detected in less than 3 cells. 

Additionally, it is important to note that for datasets with multiple batches, quality control should be performed for each sample individually as quality control thresholds can very substantially between batches. 

In [ ]:
sc.pp.filter_cells(adata, min_genes=100)
sc.pp.filter_genes(adata, min_cells=3)

### Doublet detection

As a next step, we run a doublet detection algorithm. Identifying doublets is crucial as they can lead to misclassifications or distortions in downstream analysis steps. Scanpy contains the doublet detection method Scrublet {cite}`Wolock2019`. Scrublet predicts cell doublets using a nearest-neighbor classifier of observed transcriptomes and simulated doublets. {func}`scanpy.pp.scrublet` adds `doublet_score` and `predicted_doublet` to `.obs`. One can now either filter directly on `predicted_doublet` or use the `doublet_score` later during clustering to filter clusters with high doublet scores. 

In [ ]:
%%time

sc.pp.scrublet(adata, batch_key="sample")

We can remove doublets by either filtering out the cells called as doublets, or waiting until we've done a clustering pass and filtering out any clusters with high doublet scores.

:::{seealso}
Alternative methods for doublet detection within the scverse ecosystem are [DoubletDetection](https://github.com/JonathanShor/DoubletDetection) and [SOLO](https://docs.scvi-tools.org/en/stable/user_guide/models/solo.html). You can read more about these in the [Doublet Detection chapter](https://www.sc-best-practices.org/preprocessing_visualization/quality_control.html#doublet-detection) of Single Cell Best Practices.
:::

## Normalization

The next preprocessing step is normalization. A common approach is count depth scaling with subsequent log plus one (log1p) transformation. Count depth scaling normalizes the data to a “size factor” such as the median count depth in the dataset, ten thousand (CP10k) or one million (CPM, counts per million). The size factor for count depth scaling can be controlled via `target_sum` in `pp.normalize_total`. We are applying median count depth normalization with log1p transformation (AKA log1PF).

In [ ]:
# Saving count data
adata.layers["counts"] = adata.X.copy()

In [ ]:
# Normalizing to median total counts
sc.pp.normalize_total(adata)
# Logarithmize the data
sc.pp.log1p(adata)

## Feature selection

As a next step, we want to reduce the dimensionality of the dataset and only include the most informative genes. This step is commonly known as feature selection. The scanpy function `pp.highly_variable_genes` annotates highly variable genes by reproducing the implementations of Seurat {cite}`Satija2015`, Cell Ranger {cite}`Zheng2017`, and Seurat v3 {cite}`Stuart2019` depending on the chosen `flavor`. 

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=2000, batch_key="sample")

In [ ]:
sc.pl.highly_variable_genes(adata)

with HoloViews:

In [ ]:
variable_genes_norm = hv.Points(
    adata,
    ['var.means', 'var.dispersions_norm'], 'var.highly_variable',
).opts(ylabel='dispersions of genes (normalized)')
variable_genes = hv.Points(
    adata,
    ['var.means', 'var.dispersions'], 'var.highly_variable',
).opts(ylabel='dispersions of genes (not normalized)')

(hv.Layout((variable_genes_norm, variable_genes))
 .opts(hv.opts.Points(
     color='var.highly_variable',
     cmap=['grey', 'black'],
     title='Hightly Variable Genes',
     legend_position='bottom_right',
     xlabel='mean expressions of genes',
     tools=['hover'],
     size=1,
     show_grid=True,
     alpha=.5,
 )
      )
)

## Dimensionality Reduction
Reduce the dimensionality of the data by running principal component analysis (PCA), which reveals the main axes of variation and denoises the data.

In [ ]:
sc.tl.pca(adata)

Let us inspect the contribution of single PCs to the total variance in the data. This gives us information about how many PCs we should consider in order to compute the neighborhood relations of cells, e.g. used in the clustering function {func}`~scanpy.tl.leiden` or {func}`~scanpy.tl.tsne`. In our experience, there does not seem to be signifigant downside to overestimating the numer of principal components.

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs=50, log=True)

with HoloViews:

In [ ]:
# By definition, adata.uns objects do not share dims with `adata.X`, therefore are accessed directly (e.g. as arrays) for HoloViews.

variance_ratio = adata.uns['pca']['variance_ratio']

x_positions = np.arange(1, len(variance_ratio) + 1)
labels = ['PC{}'.format(i) for i in range(1, len(variance_ratio) + 1)]

points = hv.Points((x_positions, variance_ratio), ['Ranking', 'Variance Ratio']).opts(color='Variance Ratio', cmap='magma_r', size=6, tools=['hover'])
label_overlay = hv.Labels((x_positions, variance_ratio, labels), ['x', 'y'], 'text').opts(
    text_font_size='8pt', text_align='left', yoffset=.003, angle=90, text_color='black')

var_ratio_plot = (points * label_overlay).opts(
    width=600, height=400, xlabel='Ranking', ylabel=None, title='Variance Ratio', show_grid=True)

var_ratio_plot

You can also plot the principal components to see if there are any potentially undesired features (e.g. batch, QC metrics) driving signifigant variation in this dataset. In this case, there isn't anything too alarming, but it's a good idea to explore this.

In [ ]:
sc.pl.pca(
    adata,
    color=["sample", "sample", "pct_counts_mt", "pct_counts_mt"],
    dimensions=[(0, 1), (2, 3), (0, 1), (2, 3)],
    ncols=2,
    size=2,
)

with hvPlot (convert to DataFrame):

In [ ]:
pca_df = pd.DataFrame(adata.obsm["X_pca"], columns=[f"PCA{i+1}" for i in range(adata.obsm["X_pca"].shape[-1])], index=adata.obs_names)
pca_df = pca_df.join(adata.obs)

In [ ]:
hvplot_pca_opts = dict(
    frame_width=200,
    data_aspect=1,
    tools=['hover'],
    rasterize=True,
    colorbar=False,
)

pca_p1 = pca_df.hvplot.points(
    x="PCA1",
    y="PCA2",
    aggregator=ds.count_cat('sample'),
    cmap=cc.b_glasbey_category10[:2],
    title='sample',
    **hvplot_pca_opts,
)

pca_p2 = pca_df.hvplot.points(
    x="PCA3",
    y="PCA4",
    aggregator=ds.count_cat('sample'),
    cmap=cc.b_glasbey_category10[:2],
    title='sample',
    **hvplot_pca_opts,
)

pca_p3 = pca_df.hvplot.points(
    x="PCA1",
    y="PCA2",
    color='pct_counts_mt',
    cmap='Viridis',
    title='pct_counts_mt',
    **hvplot_pca_opts,
)

pca_p4 = pca_df.hvplot.points(
    x="PCA1",
    y="PCA2",
    color='pct_counts_mt',
    cmap='Viridis',
    title='pct_counts_mt',
    **hvplot_pca_opts,
)

(pca_p1 + pca_p2 + pca_p3 + pca_p4).cols(2)

with HoloViews V1 (layout):

Since adata.obsm['X_pca'] is a 2D array, with a column per pca dim, we'll access the array data that we want directly rather than using the anndata interface.

TODO: add adata.obs and selectively pass certain vdims to view other info in hover cols

In [ ]:
import holoviews.operation.datashader as hd
import datashader as ds
import colorcet as cc
import panel as pn
import numpy as np
import holoviews as hv

pn.extension()
hv.extension('bokeh')

def create_hv_dimreduction_plot(
    x_data, color_data, x_dim, y_dim, color_var, xaxis_label, yaxis_label,
    width=300, height=300, datashading=False, labels=False,
    cont_cmap = 'Viridis',
    cat_cmap = cc.b_glasbey_category10):
    """
    Create a cluster points plot

    Parameters:
    - x_data: numpy.ndarray, shape n_obs by n_clusters
    - x_dim, y_dim: int, indices into x_data's cluster dim to use as x or y data.
    - color_data: numpy.ndarray, shape n_obs color values (categorical or continuous).
    - color_var: str, the name to give the coloring dim.
    - xaxis_label, yaxis_label: str, labels for the axes.
    - width, height: int, dimensions of the plot.
    - datashading: bool, whether to apply Datashader.
    - labels: bool, whether to overlay labels at median positions.

    """
    is_categorical = (
        color_data.dtype.name in ['category', 'categorical', 'bool'] or
        np.issubdtype(color_data.dtype, np.object_) or
        np.issubdtype(color_data.dtype, np.str_)
    )
    
    if is_categorical:
        n_unq_cat = len(np.unique(color_data))
        cmap = cat_cmap[:n_unq_cat]
        colorbar = False
        if labels:
            show_legend = False
        else:
            show_legend = True
    else:
        cmap = cont_cmap
        show_legend = False
        colorbar = True

    plot = hv.Points(
        (x_data[:, x_dim], x_data[:, y_dim], color_data),
        [xaxis_label, yaxis_label], color_var
    )

    plot_opts = dict(
        color=color_var,
        cmap=cmap,
        size=1,
        alpha=0.5,
        colorbar=colorbar,
        padding=0,
        tools=['hover'],
        show_legend=show_legend,
    )

    label_opts = dict(
        text_font_size='8pt',
        text_color='black'
    )

    if datashading:
        if is_categorical:
            aggregator = ds.count_cat(color_var)
            plot = hd.rasterize(plot, aggregator=aggregator)
            plot = hd.dynspread(plot, threshold=0.5)
            plot = plot.opts(cmap=cmap, tools=['hover'])

            if labels:
                unique_categories = np.unique(color_data)
                labels_data = []
                for cat in unique_categories:
                    mask = color_data == cat
                    median_x = np.median(x_data[mask, x_dim])
                    median_y = np.median(x_data[mask, y_dim])
                    labels_data.append((median_x, median_y, str(cat)))
                labels_element = hv.Labels(labels_data, [xaxis_label, yaxis_label], 'Label').opts(**label_opts)
                plot = plot * labels_element
            else:
                # Create a custom legend with dummy scatter plot hack
                unique_categories = np.unique(color_data)
                color_key = dict(zip(unique_categories, cmap[:len(unique_categories)]))
                legend_items = [
                    hv.Points([0,0], label=str(cat)).opts(
                        color=color_key[cat],
                        size=0
                    ) for cat in unique_categories
                ]
                legend = hv.NdOverlay({str(cat): item for cat, item in zip(unique_categories, legend_items)}).opts(
                    show_legend=True,
                    legend_position='right',
                    legend_limit=1000,
                    legend_cols= len(unique_categories) // 8,
                )
                plot = plot * legend

        else:
            aggregator = ds.mean(color_var)
            plot = hd.rasterize(plot, aggregator=aggregator)
            plot = hd.dynspread(plot, threshold=0.5)
            plot = plot.opts(cmap=cmap, colorbar=colorbar)
    else:
        plot = plot.opts(**plot_opts)
        if is_categorical and labels:
            unique_categories = np.unique(color_data)
            labels_data = []
            for cat in unique_categories:
                mask = color_data == cat
                median_x = np.median(x_data[mask, x_dim])
                median_y = np.median(x_data[mask, y_dim])
                labels_data.append((median_x, median_y, str(cat)))
            labels_element = (hv.Labels(labels_data, [xaxis_label, yaxis_label], 'Label')
                              .opts(**label_opts)
                             )
            plot = plot * labels_element

    return plot.opts(
        title=f"{color_var}",
        tools=['hover'],
        show_legend=show_legend,
        frame_width=width,
        frame_height=height
    )

def layout_dimreduction_plots(adata, color, dimensions, width=200, height=200, dim_reduction='X_pca', labels=False):
    dr_key = dim_reduction
    dr_label = dr_key.split('_')[1].upper()
    plots = []
    x_data = adata.obsm[dr_key]
    for color_var, (x_dim, y_dim) in zip(color, dimensions):
        try: # color per derived obs
            color_data = adata.obs[color_var].values
        except: # color per gene expression
            color_data = adata.X.getcol(adata.var_names.get_loc(color_var)).toarray().flatten()
        plot = create_hv_dimreduction_plot(
            x_data, color_data, x_dim, y_dim, color_var,
            f'{dr_label}{x_dim + 1}', f'{dr_label}{y_dim + 1}',
            width=width,
            height=height,
            datashading=True,
            labels=labels,
        )
        plots.append(plot)
    return hv.Layout(plots).opts(shared_axes=False, axiswise=True)

In [ ]:
layout = layout_dimreduction_plots(
    adata=adata,
    color=["sample", "sample", "pct_counts_mt", "pct_counts_mt"],
    dimensions=[(0, 1), (2, 3), (0, 1), (2, 3)],
    dim_reduction='X_pca',
    labels=False
)
layout.cols(2)

with HoloViews v2 (UI for axes and coloring):

In [ ]:
from panel.io import hold

def dimreduction_app(
    adata,
    dim_reduction=None,
    color_by=None,
    datashade=True,
    show_widgets=True,
    width=200,
    height=200,
    labels=False,
):
    
    color_options = list(adata.obs.columns)
    default_color = color_by or color_options[0]

    # Map dimension reduction methods to their labels
    dr_options = {key: key.split('_')[1].upper() for key in adata.obsm.keys()}
    default_dr = dim_reduction or list(dr_options.keys())[0]

    x_data = adata.obsm[default_dr]
    num_dims = x_data.shape[1]

    dr_label = dr_options[default_dr]
    dim_labels = [f"{dr_label}{i+1}" for i in range(num_dims)]
    dim_mapping = {label: index for index, label in enumerate(dim_labels)}

    dr_select = pn.widgets.Select(
        name='Dim Reduction', options=list(dr_options.keys()), value=default_dr
    )
    xaxis = pn.widgets.Select(name='X-axis', options=dim_labels, value=dim_labels[0])
    yaxis = pn.widgets.Select(name='Y-axis', options=dim_labels, value=dim_labels[1])
    color = pn.widgets.Select(name='Color By', options=color_options, value=default_color)
    datashading_switch = pn.widgets.Checkbox(name='Enable Datashader', value=datashade)

    def update_plot(dr_select_value, xaxis_value, yaxis_value, color_value, datashading_value):
        x_data = adata.obsm[dr_select_value]
        dr_label = dr_options[dr_select_value]
        num_dims = x_data.shape[1]
        dim_labels = [f"{dr_label}{i+1}" for i in range(num_dims)]
        dim_mapping = {label: index for index, label in enumerate(dim_labels)}

        x_dim = dim_mapping[xaxis_value]
        y_dim = dim_mapping[yaxis_value]
        color_data = adata.obs[color_value].values

        return create_hv_dimreduction_plot(
            x_data,
            color_data,
            x_dim,
            y_dim,
            color_value,
            xaxis_value,
            yaxis_value,
            width=width,
            height=height,
            datashading=datashading_value,
            labels=labels,
        )

    plot_pane = pn.bind(
        update_plot,
        dr_select_value=dr_select,
        xaxis_value=xaxis,
        yaxis_value=yaxis,
        color_value=color,
        datashading_value=datashading_switch,
    )

    @hold()
    def update_axis_options(event):
        x_data = adata.obsm[event.new]
        num_dims = x_data.shape[1]
        dr_label = dr_options[event.new]
        new_dim_labels = [f"{dr_label}{i+1}" for i in range(num_dims)]
        xaxis.options = new_dim_labels
        yaxis.options = new_dim_labels
        xaxis.value = new_dim_labels[0]
        yaxis.value = new_dim_labels[1]

    dr_select.param.watch(update_axis_options, 'value')

    # Ensure x-axis and y-axis selections are different
    def enforce_different_axes(event):
        if xaxis.value == yaxis.value:
            available_options = [opt for opt in yaxis.options if opt != xaxis.value]
            if available_options:
                yaxis.value = available_options[0]

    xaxis.param.watch(enforce_different_axes, 'value')
    yaxis.param.watch(enforce_different_axes, 'value')

    widgets = pn.WidgetBox(dr_select, xaxis, yaxis, color, datashading_switch)
    if show_widgets:
        app = pn.Row(widgets, plot_pane)
    else:
        app = pn.Row(plot_pane)
    return app


In [ ]:
dimreduction_app(adata, dim_reduction='X_pca', color_by='sample', show_widgets=True)

## Nearest neighbor graph constuction and visualization

Let us compute the neighborhood graph of cells using the PCA representation of the data matrix.

In [ ]:
sc.pp.neighbors(adata)

This graph can then be embedded in two dimensions for visualiztion with UMAP (McInnes et al., 2018):

In [ ]:
sc.tl.umap(adata)

We can now visualize the UMAP according to the `sample`. 

In [ ]:
sc.pl.umap(
    adata,
    color="sample",
    # Setting a smaller point size to prevent overlap
    size=2,
)

with HoloViews:

In [ ]:
layout = layout_dimreduction_plots(
    adata=adata,
    color=["sample"],
    dimensions=[(0, 1)],
    dim_reduction='X_umap'
)
layout

or with widgets:

In [ ]:
dimreduction_app(adata, dim_reduction='X_umap')

Even though the data considered in this tutorial includes two different samples, we only observe a minor batch effect and we can continue with clustering and annotation of our data. 

If you inspect batch effects in your UMAP it can be beneficial to integrate across samples and perform batch correction/integration. We recommend checking out [`scanorama`](https://github.com/brianhie/scanorama) and [`scvi-tools`](https://scvi-tools.org) for batch integration.

## Clustering

As with Seurat and many other frameworks, we recommend the Leiden graph-clustering method (community detection based on optimizing modularity) {cite}`Traag2019`. Note that Leiden clustering directly clusters the neighborhood graph of cells, which we already computed in the previous section.

In [ ]:
%%time

# Using the igraph implementation and a fixed number of iterations can be significantly faster, especially for larger datasets
sc.tl.leiden(adata, flavor="igraph", n_iterations=2)

In [ ]:
sc.pl.umap(adata, color=["leiden"])

with HoloViews:

In [ ]:
dimreduction_app(adata, dim_reduction='X_umap', color_by='leiden')

## Re-assess quality control and cell filtering 

As indicated before, we will now re-assess our filtering strategy by visualizing different QC metrics using UMAP. 

In [ ]:
sc.pl.umap(
    adata,
    color=["leiden", "predicted_doublet", "doublet_score"],
    # increase horizontal space between panels
    wspace=0.5,
    size=3,
)

In [ ]:
layout = layout_dimreduction_plots(
    adata=adata,
    color=["leiden", "predicted_doublet", "doublet_score"],
    dimensions=[(0, 1)] *3,
    dim_reduction='X_umap',
)
layout

In [ ]:
sc.pl.umap(
    adata,
    color=["leiden", "log1p_total_counts", "pct_counts_mt", "log1p_n_genes_by_counts"],
    wspace=0.5,
    ncols=2,
)

In [ ]:
layout = layout_dimreduction_plots(
    adata=adata,
    color=["leiden", "log1p_total_counts", "pct_counts_mt", "log1p_n_genes_by_counts"],
    dimensions=[(0, 1)]*4,
    dim_reduction='X_umap',
)
layout.cols(2)

## Manual cell-type annotation

:::{note}
This section of the tutorial is expanded upon using prior knowledge resources like automated assignment and gene enrichment in the scverse tutorial [here](https://scverse-tutorials.readthedocs.io/en/latest/notebooks/basic-scrna-tutorial.html#cell-type-annotation)
:::

Cell type annotation is laborous and repetitive task, one which typically requires multiple rounds of subclustering and re-annotation. It's difficult to show the entirety of the process in this tutorial, but we aim to show how the tools scanpy provides assist in this process.

We have now reached a point where we have obtained a set of cells with decent quality, and we can proceed to their annotation to known cell types. Typically, this is done using genes that are exclusively expressed by a given cell type, or in other words these genes are the marker genes of the cell types, and are thus used to distinguish the heterogeneous groups of cells in our data. Previous efforts have collected and curated various marker genes into available resources, such as [CellMarker](http://bio-bigdata.hrbmu.edu.cn/CellMarker/), [TF-Marker](http://bio.liclab.net/TF-Marker/), and [PanglaoDB](https://panglaodb.se/). The [cellxgene gene expression tool](https://cellxgene.cziscience.com/gene-expression) can also be quite useful to see which cell types a gene has been expressed in across many existing datasets.

Commonly and classically, cell type annotation uses those marker genes subsequent to the grouping of the cells into clusters. So, let's generate a set of clustering solutions which we can then use to annotate our cell types. Here, we will use the Leiden clustering algorithm which will extract cell communities from our nearest neighbours graph.

In [ ]:
for res in [0.02, 0.5, 2.0]:
    sc.tl.leiden(
        adata, key_added=f"leiden_res_{res:4.2f}", resolution=res, flavor="igraph"
    )

Notably, the number of clusters that we define is largely arbitrary, and so is the `resolution` parameter that we use to control for it. As such, the number of clusters is ultimately bound to the stable and biologically-meaningful groups that we can ultimately distringuish, typically done by experts in the corresponding field or by using expert-curated prior knowledge in the form of markers.

In [ ]:
sc.pl.umap(
    adata,
    color=["leiden_res_0.02", "leiden_res_0.50", "leiden_res_2.00"],
    legend_loc="on data",
)

In [ ]:
layout = layout_dimreduction_plots(
    adata=adata,
    color=["leiden_res_0.02", "leiden_res_0.50", "leiden_res_2.00"],
    dimensions=[(0, 1)]*3,
    dim_reduction='X_umap',
    labels=True,
)
layout

Though UMAPs should not be over-interpreted, here we can already see that in the highest resolution our data is over-clustered, while the lowest resolution is likely grouping cells which belong to distinct cell identities.

### Marker gene set

Let's define a set of marker genes for the main cell types that we expect to see in this dataset. These were adapted from [Single Cell Best Practices annotation chapter](https://www.sc-best-practices.org/cellular_structure/annotation.html), for a more detailed overview and best practices in cell type annotation, we refer the user to it.

In [ ]:
marker_genes = {
    "CD14+ Mono": ["FCN1", "CD14"],
    "CD16+ Mono": ["TCF7L2", "FCGR3A", "LYN"],
    # Note: DMXL2 should be negative
    "cDC2": ["CST3", "COTL1", "LYZ", "DMXL2", "CLEC10A", "FCER1A"],
    "Erythroblast": ["MKI67", "HBA1", "HBB"],
    # Note HBM and GYPA are negative markers
    "Proerythroblast": ["CDK6", "SYNGR1", "HBM", "GYPA"],
    "NK": ["GNLY", "NKG7", "CD247", "FCER1G", "TYROBP", "KLRG1", "FCGR3A"],
    "ILC": ["ID2", "PLCG2", "GNLY", "SYNE1"],
    "Naive CD20+ B": ["MS4A1", "IL4R", "IGHD", "FCRL1", "IGHM"],
    # Note IGHD and IGHM are negative markers
    "B cells": [
        "MS4A1",
        "ITGB1",
        "COL4A4",
        "PRDM1",
        "IRF4",
        "PAX5",
        "BCL11A",
        "BLK",
        "IGHD",
        "IGHM",
    ],
    "Plasma cells": ["MZB1", "HSP90B1", "FNDC3B", "PRDM1", "IGKC", "JCHAIN"],
    # Note PAX5 is a negative marker
    "Plasmablast": ["XBP1", "PRDM1", "PAX5"],
    "CD4+ T": ["CD4", "IL7R", "TRBC2"],
    "CD8+ T": ["CD8A", "CD8B", "GZMK", "GZMA", "CCL5", "GZMB", "GZMH", "GZMA"],
    "T naive": ["LEF1", "CCR7", "TCF7"],
    "pDC": ["GZMB", "IL3RA", "COBLL1", "TCF4"],
}

In [ ]:
sc.pl.dotplot(adata, marker_genes, groupby="leiden_res_0.02", standard_scale="var")

with HoloViews:

In [ ]:
hvc.DotPlot(
    adata=adata,
    marker_genes=marker_genes,
    groupby='leiden_res_0.02',
    max_dot_size=10,
)

In [ ]:
# TODO: add hierarchical ticks
# TODO: add dot size legend

There are fairly clear patterns of expression for our markers show here, which we can use to label our coarsest clustering with broad lineages.

In [ ]:
adata.obs["cell_type_lvl1"] = adata.obs["leiden_res_0.02"].map(
    {
        "0": "Lymphocytes",
        "1": "Monocytes",
        "2": "Erythroid",
        "3": "B Cells",
    }
)

In [ ]:
sc.pl.dotplot(adata, marker_genes, groupby="leiden_res_0.50", standard_scale="var")

In [ ]:
hvc.DotPlot(
    adata=adata,
    marker_genes=marker_genes,
    groupby="cell_type_lvl1",
    max_dot_size=10,
).opts(height=150, width=1200, responsive=False)

This seems like a resolution that suitable to distinguish most of the different cell types in our data. As such, let's try to annotate those by manually using the dotplot above, together with the UMAP of our clusters. Ideally, one would also look specifically into each cluster, and attempt to subcluster those if required.

### Differentially-expressed Genes as Markers

Furthermore, one can also calculate marker genes per cluster and then look up whether we can link those marker genes to any known biology, such as cell types and/or states. This is typically done using simple statistical tests, such as Wilcoxon and t-test, for each cluster vs the rest.

In [ ]:
# Obtain cluster-specific differentially expressed genes
sc.tl.rank_genes_groups(adata, groupby="leiden_res_0.50", method="wilcoxon")

We can then visualize the top 5 differentially-expressed genes on a dotplot.

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata, groupby="leiden_res_0.50", standard_scale="var", n_genes=5
)

with holoviz:

In [ ]:
# TODO: add dotplot functionality for using `rank_genes_groups_df` to get top n DE genes per leiden cluster
# TODO: add dendrogram

We can then use these genes to figure out what cell types we're looking at. For example, Cluster 7 is expressing [*NKG7*](https://www.genecards.org/cgi-bin/carddisp.pl?gene=NKG7&keywords=nkg7) and [*GNLY*](https://www.genecards.org/cgi-bin/carddisp.pl?gene=GNLY&keywords=GNLY), suggesting these are [NK cells](https://en.wikipedia.org/wiki/Natural_killer_cell).

To create your own plots, or use a more automated approach, the differentially expressed genes can be extracted in a convenient format with {func}`scanpy.get.rank_genes_groups_df`

In [ ]:
sc.get.rank_genes_groups_df(adata, group="7").head(5)

In [ ]:
dc_cluster_genes = sc.get.rank_genes_groups_df(adata, group="7").head(5)["names"]
sc.pl.umap(
    adata,
    color=[*dc_cluster_genes, "leiden_res_0.50"],
    legend_loc="on data",
    frameon=False,
    ncols=3,
)

In [ ]:
layout = layout_dimreduction_plots(
    adata=adata,
    color=[*dc_cluster_genes, "leiden_res_0.50"],
    dimensions=[(0, 1)]*6,
    dim_reduction='X_umap',
    labels=True,
)
layout.cols(3)

You may have noticed that the p-values found here are extremely low. This is due to the statistical test being performed considering each cell as an independent sample. For a more conservative approach you may want to consider "pseudo-bulking" your data by sample (*e.g.* `sc.get.aggregate(adata, by=["sample", "cell_type"], func="sum", layer="counts")`) and using a more powerful differential expression tool, like [`pydeseq2`](https://pydeseq2.readthedocs.io/).